In [ ]:
#kedamawi Mulualem
#mapping project


import cv2
import numpy as np
import operator
from collections import *
from tkinter import *
import networkx as nx
from operator import*

#node class for each city on the map
class Node:
    def __init__(self,initdata,close):
        self.data = initdata
        self.close=close
        #the other two naibours
        
    def getData(self):
        return self.data

    def getNext(self,i):
        if i>=len(close):
            return -1
        return self.close[i][0]
    
    
    def getNextdis(self,i):
        return self.close[i][1]
        if i>=len(close):
            return -1

    def setData(self,newdata):
        self.data = newdata

def pagerank(G, alpha=0.85):
        max_iter=100
        tol=1.0e-6
        nstart=None
        weight='weight'
        dangling=None
        if len(G) == 0:
                return {}

        if not G.is_directed():
                D = G.to_directed()
        else:
                D = G

        # Create a copy in (right) stochastic form
        W = nx.stochastic_graph(D, weight=weight)
        N = W.number_of_nodes()

        # Choose fixed starting vector if not given
        if nstart is None:
                x = dict.fromkeys(W, 1.0 / N)
        else:
                # Normalized nstart vector
                s = float(sum(nstart.values()))
                x = dict((k, v / s) for k, v in nstart.items())
        for _ in range(max_iter):
                xlast = x
                x = dict.fromkeys(xlast.keys(), 0)
                danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
                for n in x:

                        # this matrix multiply looks odd because it is
                        # doing a left multiply x^T=xlast^T*W
                        for nbr in W[n]:
                                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
                        x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

                # check convergence, l1 norm
                err = sum([abs(x[n] - xlast[n]) for n in x])
                if err < N*tol:
                        return x

def transparentOverlay(src , overlay , pos=(0,0),scale = 0.5):
    overlay = cv2.resize(overlay,(0,0),fx=scale,fy=scale)
    h,w,_ = overlay.shape  # Size of foreground
    rows,cols,_ = src.shape  # Size of background Image
    y,x = pos[0],pos[1]    # Position of foreground/overlay image
    
    #loop over all pixels and apply the blending equation
    for i in range(h):
        for j in range(w):
            if x+i >= rows or y+j >= cols:
                continue
            alpha = float(overlay[i][j][3]/255.0) # read the alpha channel 
            src[x+i][y+j] = alpha*overlay[i][j][:3]+(1-alpha)*src[x+i][y+j]
    return src
#a draw line function that takes image and two points and drow a line from the first point to the second
def drawline(img,a,b):
    x, y = a[0]
    x = int(x)
    y = int(y)
    
    x2, y2 = b[0]
    x2 = int(x2)
    y2 = int(y2)
    cv2.line(img,(x,y),(x2,y2),(120,1,0),2)
#this function is used to detect the dotes from the image and assighn them a value
def get_points(inputpic):
    img = cv2.imread(inputpic)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    corners = cv2.goodFeaturesToTrack(gray, 48, 0.5,0.95)
    return corners


#this function is used to calculet the eculidean distance between two points 
def distance(a,b):
    x, y = a[0]
    x = int(x)
    y = int(y)
    
    x2, y2 = b[0]
    x2 = int(x2)
    y2 = int(y2)
    if x is x2 and y is y2:
        return 0
    answer=((x-x2)**2)+((y-y2)**2)
    answer=(answer)**0.5
    return answer

#font for ths text
font = cv2.FONT_HERSHEY_SIMPLEX
#default cloth citys 
amount=5
#graphe to store the index to pixcel value
graph={}  
#name of the input image
mapinput="usadoted.jpg"
#reading the image
image3=cv2.imread(mapinput,1)
#reading image with points
corners=get_points("kd.jpg")

#array to save clothest arryay
close=[]
def setupmatrix(amount):
    global graph
    global close

    image= cv2.imread(mapinput,0)
    #for path
    global image3
    global corners
 #for each dot detected
    for i,corner in enumerate(corners):
        close=[]
        #find the dixtance of all other citys from the current city and add it to close array
        for j,cornerss in enumerate(corners):      
            if j is not i:
                close.append([j,distance(corner,cornerss)])

        #sort the array by the distance of each city
        close= sorted(close, key=lambda x: x[1])    
        #take n  amount of closest citys which will be used to generate the graph
        close=close[:amount]
        #create a node for the current city and addit to the graph which have all the other citys.
        graph[i]=Node(i,close)

    counter =0
    #put id to each dot on the image for visualization 
    for i in graph:
        x=corners[counter][0][0]
        y= corners[counter][0][1]
        cv2.putText(image3,str(counter),(x,y), font, 0.6,(100,150,200),1,cv2.LINE_AA)

        counter+=1
        mtx=[]
    #generate the adjacency matrix
    for i in range(len(graph)):
        a=[]
        for j in range(len(graph)):
                a.append(0)
        for j in range(amount):
            a[graph[i].getNext(j)]=graph[i].getNextdis(j)
        mtx.append(a)

    global mat
    mat=mtx
    #making it two way road
    for c,r in enumerate(mtx):
        for cc ,rr in enumerate(r):
            if rr is not 0:

                mtx[cc][c]=rr
    return mtx


city={}
city[12]="Alabama"
city[20]="Arizona"
city[29]="Arkansas"
city[21]="California"
city[8]="Colorado"
city[5]="Connecticut"
city[3]="Delaware"
city[10]="Florida"
city[17]="Georgia"
city[9]="Idaho" 
city[0]="Illinois"
city[41]="Indiana"
city[32]="Iowa"
city[11]="Kansas"
city[13]="Kentucky"
city[40]="Louisiana"
city[38]="Maine"
city[27]="Maryland"
city[42]="Massachusetts"
city[43]="Michigan"
city[28]="Minnesota"
city[16]="Mississippi"
city[35]="Missouri"
city[4]="Montana"
city[33]="Nebraska"
city[46]="Nevada"
city[31]="New Hampshire"
city[23]="New Jersey"
city[30]="New Mexico"
city[37]="New York"
city[36]="North Carolina"
city[26]="North Dakota"
city[18]="Ohio"
city[47]="Oklahoma"
city[2]="Oregon"
city[14]="Pennsylvania"
city[39]="Rhode Island"
city[6]="South Carolina"
city[25]="South Dakota"
city[22]="Tennessee"
city[19]="Texas"
city[7]="Utah"
city[24]="Vermont"
city[15]="Virginia"
city[44]="Washington"
city[45]="West Virginia"
city[34]="Wisconsin"
city[1]="Wyoming"
index={}
index["Alabama"]=12
index["Arizona"]=20
index["Arkansas"]=29
index["California"]=21
index["Colorado"]=8
index["Connecticut"]=5
index["Delaware"]=3
index["Florida"]=10
index["Georgia"]=17
index["Idaho" ]=9
index["Illinois"]=0
index["Indiana"]=41
index["Iowa"]=32
index["Kansas"]=11
index["Kentucky"]=13
index["Louisiana"]=40
index["Maine"]=38
index["Maryland"]=27
index["Massachusetts"]=42
index["Michigan"]=43
index["Minnesota"]=28
index["Mississippi"]=16
index["Missouri"]=35
index["Montana"]=4
index["Nebraska"]=33
index["Nevada"]=46
index["New Hampshire"]=31
index["New Jersey"]=23
index["New Mexico"]=30
index["New York"]=37
index["North Carolina"]=36
index["North Dakota"]=26
index["Ohio"]=18
index["Oklahoma"]=47
index["Oregon"]=2
index["Pennsylvania"]=14
index["Rhode Island"]=39
index["South Carolina"]=6
index["South Dakota"]=25
index["Tennessee"]=22
index["Texas"]=19
index["Utah"]=7
index["Vermont"]=24
index["Virginia"]=15
index["Washington"]=44
index["West Virginia"]=45
index["Wisconsin"]=34
index["Wyoming"]=1



class Graph:
        a=[]
        def smallestDistance(self,distance,queue):
                minimum = float("Inf")
                min_index = -1
                #from the dist array,pick one which has min value and is till in queue
                for i in range(len(distance)):
                        if distance[i] < minimum and i in queue:
                                minimum = distance[i]
                                min_index = i
                return min_index

        def Path(self, parent, j):
                      
                if parent[j] == -1 : #Base Case : If j is source
                        Graph.a.append(j)
                        return
                self.Path(parent , parent[j])
                Graph.a.append(j)



        def Solution(self, dist, parent,final):
                src =0
                a=self.Path(parent,final)
                return a



        def dijkstra(self, graph, src,final):

                row = len(graph)
                col = len(graph[0])
                dist = [float("Inf")] * row

                #Parent array to store shortest path tree
                parent = [-1] * row

                # Distance of source vertex from itself is always 0
                dist[src] = 0
        
                # Add all vertices in queue
                queue = []
                for i in range(row):
                        queue.append(i)
                        
                #Find shortest path for all vertices
                while queue:

                        u = self.smallestDistance(dist,queue) 

                        queue.remove(u)
                        for i in range(col):
                                if graph[u][i] and i in queue:
                                        if dist[u] + graph[u][i] < dist[i]:
                                                dist[i] = dist[u] + graph[u][i]
                                                parent[i] = u
                answer=self.Solution(dist,parent,final)
                

g= Graph()


#Print the solution
path=[]

start=1
end=1

def rungui():

    # Dictionary with options
    choices = [
    "Alabama",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming"]

    path=[0,0]

    def showmap(amount):
        global graph
        global close

        image3=cv2.imread("usadoted.jpg",1)
        #for path
        global corners
        for i,corner in enumerate(corners):
            close=[]

            for j,cornerss in enumerate(corners):      
                if j is not i:
                    close.append([j,distance(corner,cornerss)])


            close= sorted(close, key=lambda x: x[1])    
            close=close[:amount]

            graph[i]=Node(i,close)
  
        counter =0

        for i in graph:
            x=corners[counter][0][0]
            y= corners[counter][0][1]
            cv2.putText(image3,str(counter),(x,y), font, 0.6,(100,10,200),1,cv2.LINE_AA)

            counter+=1
            for j in range(amount):
                drawline(image3,corners[graph[i].getData()],corners[graph[i].getNext(j)])
        cv2.imshow("USA MAP", image3)
        cv2.waitKey()
        cv2.destroyAllWindows()
    def calculete():
        global start
        global end
        
        if start is not 0 and end is not 0:
            #set the path array to empty
            g.a[:] = [] 
            g.dijkstra(setupmatrix(w1.get()),start,end)
            path1=[]
            path1=g.a

        drawpath(path1)
    def clickAbout():
        toplevel = Toplevel()
        height = 40
        width = 1
        setupmatrix(w1.get())
        A = np.array(mat)
        G = nx.DiGraph(A)
        pr=nx.pagerank(G,0.4)
        kd= OrderedDict(sorted(pr.items(), key=itemgetter(1), reverse=True))
        counter=0;
        count=0
        b = Label(toplevel, text="By conecting each city to the closest "+str(w1.get())+" citys")
        b.grid(row=0, column=2)
        for i in kd: #Rows
            b = Label(toplevel, text=str(counter+count+1)+"."+city[i]+"        "+str(kd[i]))
            if counter <= 24:
                b.grid(row=counter+1, column=0)
                counter+=1
            else:
                b.grid(row=count+1, column=3)
                count+=1
            

        
       

    def drawpath(path):
        image2=cv2.imread("usadoted.jpg",1) 
        #png images which will be used to represent the start and end points.
        overlayImage1 = cv2.imread("loc1.png" , cv2.IMREAD_UNCHANGED)
        overlayImage2 = cv2.imread("locm.png" , cv2.IMREAD_UNCHANGED)
        #draw line between the cities on the path array
        for j ,i in enumerate(path):
            k=int(corners[i][0][0])
            m=int(corners[i][0][1])  
            #write the name of the state of the city.
            cv2.putText(image2,city.get(i),(k,m), font, 0.6,(100,150,200),1,cv2.LINE_AA)
            if j<len(path)-1:
                drawline(image2,corners[path[j]],corners[path[j+1]])

        #add the start png sign on the image 
        x, y = corners[path[0]][0]
        x = int(x)
        y = int(y)
        cv2.rectangle(image2,(x-1,y-1),(x+1,y+1),(0,190,0), 9)
        image2 = transparentOverlay(image2,overlayImage2,(x-30,y-48),0.10)
        

       #add the end png sign on the image 
        x, y = corners[path[len(path)-1]][0]
        x = int(x)
        y = int(y)
        cv2.rectangle(image2,(x-1,y-1),(x+1,y+1),(100,255,0), 9)
        image2 = transparentOverlay(image2,overlayImage1,(x-12,y-25),0.02)
        #show image
        cv2.imshow("USA MAP ", image2)
        cv2.waitKey()
        cv2.destroyAllWindows()

    root = Tk()
    root.title("KD Mapping")

    # Add a grid
    mainframe = Frame(root)
    mainframe.grid(column=0,row=0, sticky=(N,W,E,S) )
    mainframe.columnconfigure(0, weight = 1)
    mainframe.rowconfigure(0, weight = 1)
    mainframe.pack(pady = 100, padx = 100)

    # Create a Tkinter variable
    tkvar1 = StringVar(root)
    tkvar2 = StringVar(root)
    tkvar1.set("State 1")
    tkvar2.set("State 2")# set the default option
    popupMenu1 = OptionMenu(mainframe, tkvar1, *choices)


    popupMenu2 = OptionMenu(mainframe, tkvar2, *choices)
    Label(mainframe, text="Choose starting city").grid(row = 0, column = 1)
    popupMenu1.grid(row = 2, column =1)
    Label(mainframe, text="Choose destination").grid(row = 3, column = 1)
    popupMenu2.grid(row = 4, column =1)
    w1 = Scale(mainframe, from_=4, to=47,length=300, tickinterval=4,orient=HORIZONTAL)
    w1.grid(row=6, column =1)
    def cal():
        showmap(w1.get())
    getpath = Button(mainframe, text="Get path", command=calculete)
    getpath.grid(row = 5, column =0)
    
    getrank = Button(mainframe, text="Get rank", command=clickAbout)
    getrank.grid(row = 5, column =1)
    
    getmap = Button(mainframe, text="Get map", command=cal)
    getmap.grid(row = 5, column =2)




    # on change dropdown value
    start=0
    end=0
    def change_dropdown1(*args):
        global start
        search=tkvar1.get()
        start=index[search]

    def change_dropdown2(*args):
        global end
        search=tkvar2.get()
        end=index[search]

    # link function to change dropdown
    tkvar1.trace('w',change_dropdown1)
    tkvar2.trace('w',change_dropdown2)
    root.mainloop()

rungui()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\kedam\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-1-c59192ddefe4>", line 462, in calculete
    drawpath(path1)
  File "<ipython-input-1-c59192ddefe4>", line 508, in drawpath
    image2 = transparentOverlay(image2,overlayImage2,(x-30,y-48),0.10)
  File "<ipython-input-1-c59192ddefe4>", line 80, in transparentOverlay
    overlay = cv2.resize(overlay,(0,0),fx=scale,fy=scale)
cv2.error: OpenCV(3.4.3) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

